## Loading data from outside EASI OpenDataCube

### NetCDF using OPeNDAP THREDDS

In [ ]:
import s3fs
import xarray as xr

In [ ]:
# BARRA2_THREDDS_CATALOG = "https://thredds.nci.org.au/thredds/catalog/catalogs/ob53/catalog.html"

In [ ]:
# Can use links to OPeNDAP NetCDF files
BARRA2_THREDDS_URL = "https://thredds.nci.org.au/thredds/dodsC/ob53/output/reanalysis/AUST-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/CAPE/latest/CAPE_AUST-11_ERA5_historical_hres_BOM_BARRA-R2_v1_day_202505-202505.nc"

In [ ]:
ds = xr.open_dataset(BARRA2_THREDDS_URL, chunks={"time": 1, "lat": 100, "lon": 100})
# Or for automatic chunking:
# ds = xr.open_dataset(BARRA2_THREDDS_URL, chunks="auto")

In [ ]:
ds

In [ ]:
ds_subset = ds.isel(time=0)

In [ ]:
ds_subset

In [ ]:
ds_subset = ds_subset.compute()

In [ ]:
ds_subset

In [ ]:
ds_subset["CAPE"].data

### NetCDF from S3

In [ ]:
ERA5_BUCKET = "nsf-ncar-era5"
ERA5_KEY = "e5.oper.fc.sfc.minmax/202506/e5.oper.fc.sfc.minmax.228_227_mntpr.ll025sc.2025061606_2025070106.nc"

In [ ]:
storage_options = {
    "anon": True,            # or True if the bucket is public
    "client_kwargs": {
        "region_name": "us-west-2"  # Change this to your desired region
    }
}

# Initialize S3FileSystem (use anon=True for public buckets)
fs_s3 = s3fs.S3FileSystem(**storage_options)

# Define the path to your NetCDF file on S3
s3_path = f"{ERA5_BUCKET}/{ERA5_KEY}"

# Open the remote file as a file-like object
remote_file_obj = fs_s3.open(s3_path, mode="rb")

# Open the dataset with xarray, enabling lazy loading
ds = xr.open_dataset(remote_file_obj, chunks="auto", engine="h5netcdf")

In [ ]:
ds

In [ ]:
ds_subset = ds.isel(forecast_initial_time=0, forecast_hour=0)["MNTPR"].compute()

In [ ]:
ds_subset